In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from collections import Counter

import pandas as pd

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


В качестве готовой модели была взята модель "Tatyana/rubert-base-cased-sentiment-new".

In [3]:
from transformers import pipeline

sentiment = pipeline("text-classification", model='Tatyana/rubert-base-cased-sentiment-new')
sentiment("Этот ресторан отличный")

[{'label': 'POSITIVE', 'score': 0.9541073441505432}]

In [4]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")
df_val = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/val.csv")

df_train.shape, df_val.shape

((181467, 3), (22683, 3))

In [5]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [6]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

In [7]:
idx = 11
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
sentiment(df_train.iloc[idx]['text'])

мартовские путёвки дорожают на глазах. только пару дней назад были за 66, уже 86 о_О
label is 0


[{'label': 'NEGATIVE', 'score': 0.9948994517326355}]

In [8]:
df_train['text'] = df_train['text'].apply(lambda x: x.lower())
df_val['text'] = df_val['text'].apply(lambda x: x.lower())

In [9]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('Tatyana/rubert-base-cased-sentiment-new')

example_text = 'Пример текста для токенизации'
bert_input = tokenizer(example_text, padding='max_length', max_length=10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

tensor([[  101, 17863, 10316,   949, 15427,   831,  2703,   102,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [10]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('Tatyana/rubert-base-cased-sentiment-new')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [11]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [12]:
from torch import nn
from transformers import BertModel


class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.bert = BertModel.from_pretrained('Tatyana/rubert-base-cased-sentiment-new')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        
        _, pooled_output = self.bert(input_ids=x, attention_mask=mask, return_dict=False)
        # _, pooled_output - набор эмбеддинигов слов, эмбеддинг предложения
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)
        return final_layer

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [14]:
model = BertClassifier().to(device)
criterion = nn.CrossEntropyLoss()

optimizer = Adam(model.linear.parameters(), lr=0.001)  # неполное обучение

Some weights of the model checkpoint at Tatyana/rubert-base-cased-sentiment-new were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Получение метрик на валидационной выборке:

In [15]:
for epoch_num in range(1):
    total_acc_train = 0
    total_loss_train = 0
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc
            
    print(
        f'Epochs: {epoch_num + 1} |  \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

Epochs: 1 |          | Val Loss:  0.011         | Val Accuracy:  0.505


Дообучение модели и получение метрик:

In [16]:
for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)
                
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

100%|██████████| 2836/2836 [05:19<00:00,  8.88it/s]


Epochs: 1 | Train Loss:  0.010         | Train Accuracy:  0.587         | Val Loss:  0.010         | Val Accuracy:  0.584


100%|██████████| 2836/2836 [05:18<00:00,  8.89it/s]


Epochs: 2 | Train Loss:  0.010         | Train Accuracy:  0.587         | Val Loss:  0.010         | Val Accuracy:  0.585


Как видно из метрик, дообученная модель имеет более лучшие показатели Val Loss и Val Accuracy, качество которых с увеличением эпох, вероятно, будет расти.